## a notebook to keep track of bugs I solved within coffea that gave me more insight into what I'm doing (note: the code will not run in this notebook, I'm just copy and pasting it as an example)

### subsection: cuts

I noticed when trying to apply preselection cuts two different ways (that are supposed to be equivalent) I was getting different results:

In [ ]:
electron_cuts = (events.Electron.pt >= 5) & (np.abs(events.Electron.eta) < 2.4)

event_mask = ak.any(electron_cuts, axis=1)

selected_events_1 = events[event_mask]

print(f"Event count after cuts: {len(selected_events_1)}")

And:

In [ ]:
from coffea.analysis_tools import PackedSelection

selection = PackedSelection()

selection.add("e pt", ak.any(events.Electron.pt >= 5, axis=1))
selection.add("e eta", ak.any(np.abs(events.Electron.eta) < 2.4, axis=1))

selected_events_2 = events[selection.all("e pt", "e eta")]

print(f"Event count after cuts: {len(selected_events_2)}")

the two code blocks (for the TSlepSlep root file I was working with) would give slightly different print outs:

`Event count after cuts: 680656`

and

`Event count after cuts: 680659`

After many confused hours, the answer is that packed selection was applying each selection one at a time to each electron, so for example you could have an event where one electron satisfies the first pt cut and another electron satisfies the eta cut and thus the expression would evaluate to true and "pass the cuts", however this is not what we want. I want passing events to have at least one electron that **itself** passes all the cuts. This being the case, the solution was to rewrite the packed selection code block like so:

In [ ]:
from coffea.analysis_tools import PackedSelection

selection = PackedSelection()

selection.add("e selection", ak.any((events.Electron.pt >= 5) & (np.abs(events.Electron.eta) < 2.4), axis=1))
selected_events_2 = events[selection.all("e selection")]

print(f"Event count after cuts: {len(selected_events_2)}")

And finally, this block will yield the correct output: `Event count after cuts: 680656`

And here is proof that this is the correct answer by ground truthing it with root interactive mode:

![ground truthed cuts](./notebook_pics/root_cuts_gt.png)


---